0928chatbot에서 픽클 저장후 불러온 픽클로 돌리기

!pip는 ipynb에서 설치때문에

In [ ]:
!pip install numpy
!pip install nltk
!pip install tensorflow
!pip install tflearn

아래 import만 있음 됨

In [46]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random

import pickle
import json
from tensorflow.python.framework import ops
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#### 집밥 json 데이터를 data로 로드시키기

In [48]:
with open('Zipbob_chatbotdata.json' ,encoding='UTF8') as file:
    data = json.load(file)

#### 집밥 pickle 데이터를 열고 words,labels... 으로 불러오기

In [40]:
  with open("Zipbobchat.pickle", "rb") as f:
        words, labels, training, output=pickle.load(f)

#### 아래 코드는 챗봇 구현 코드

In [42]:
ops.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

try:
    model.load("model.tflearn")
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [43]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))


#### 동작 완료

In [49]:
chat()

Start talking with the bot (type quit to stop)!
You: 하이
무슨 일인가요?
You: quit
